In [2]:
import sys
import numpy as np
import pandas as pd

In [3]:
# address of the file containing the interaction between playlists and tracks (train.csv)
train_file = '../data/train.csv'

In [4]:
# acquiring data from train
train_data = pd.read_csv(train_file)
train_data.columns = ['playlist_id', 'track_id']

In [5]:
# just checking the structure of the variable
np.shape(train_data)

(115553, 2)

In [6]:
tracks = train_data['track_id']

In [7]:
# just checking the structure of the variable
np.shape(tracks)


(115553,)

In [9]:
# creation of a dictionary 'track:occurrencies'
temporary_chart = {}
for track in tracks:
    if track not in temporary_chart:
        temporary_chart[track] = 1
    else:
        temporary_chart[track] += 1

In [10]:
final_chart = sorted(temporary_chart.items() ,  key=lambda x: x[1], reverse=True)

In [12]:
#printing the global top-10
for i in range(10):
    print(final_chart[i])

(8956, 1785)
(10848, 1725)
(5606, 1666)
(15578, 1579)
(10496, 1425)
(17239, 1394)
(2674, 1392)
(13980, 1379)
(18266, 1302)
(2272, 1300)


In [13]:
# address of the file containing all playlists (target_playlists.csv)
target_playlists_file = '../data/target_playlists.csv'

In [14]:
target_playlists = pd.read_csv(target_playlists_file)
target_playlists.columns = ['playlist_id']

In [16]:
# selection of the target playlists with songs
useful_data = pd.merge(target_playlists, train_data, on='playlist_id', how='inner')

In [20]:
# creating a dictionary containing playlist:list of songs of that playlist
raw_playlists = {}
for row in useful_data.itertuples(index=True, name='Pandas'):
    playlist = getattr(row, "playlist_id")
    track = getattr(row, "track_id")
    if playlist not in raw_playlists:
        raw_playlists[playlist] = [track]
    else:
        raw_playlists[playlist].append(track)

In [23]:
# useful to get the element of the tuple that is useful for the creation of the list of songs
def extract_song_id(position):
    return position[0]

In [24]:
file = open("submission.csv", 'a')
file.write("playlist_id,track_ids" + '\n')

# useful to print to file with the right structure (funzione pessima ma non capisco più niente, rivedere)
def print_to_file(playlist, tracks, file):
    file.write(str(playlist) + ',')
    index = 0
    while index < 9:
        file.write(str(tracks[index]) + ' ')
        index += 1
    file.write(str(tracks[index]) + '\n')  


In [25]:
# this function provide the recommendation after receiving tracks of the playlist and the chart
def recommend(playlist, tracks, chart):
    '''
    it provides recommendations for a playlist, based on the tracks present in the playlist and based on the global chart
    '''
    to_be_recommended = []
    count = 0
    index = 0
    while count < 10:
        if extract_song_id(chart[index]) not in tracks:
            to_be_recommended.append(extract_song_id(chart[index]))
            count += 1
        index +=1
    print_to_file(playlist, to_be_recommended, file)

In [26]:
for playlist in raw_playlists:
    recommend(playlist, raw_playlists[playlist], final_chart)
file.close()